# Euros 2024: Fetch teams and team details

#### Load Python tools and Jupyter config

In [1]:
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
today = pd.Timestamp("today").strftime("%Y%m%d")

---

## Read data

#### Headers

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    "x-api-key": "ceeee1a5bb209502c6c438abd8f30aef179ce669bb9288f2d1cf2fa276de03f4",
}

In [ ]:
params = {
    "competitionId": "3",
    "limit": "55", 
    "offset": "0", 
    "phase": "TOURNAMENT", # final phase (vs. 'ALL')
    "seasonYear": "2024",
}

In [ ]:
response = requests.get(
    "https://comp.uefa.com/v2/teams", params=params, headers=headers
)

In [42]:
teams_df = pd.DataFrame(response.json())[
    [
        "associationId",
        "internationalName",
        "teamCode",
    ]
]

In [45]:
teams_df.head(100)

,associationId,internationalName,teamCode
0,200109,Poland,POL
1,200110,Portugal,POR
2,200113,Romania,ROU
3,200117,Scotland,SCO
4,200122,Spain,ESP
5,200128,Switzerland,SUI
6,200002,Albania,ALB
7,200135,Türkiye,TUR
8,200008,Austria,AUT
9,200013,Belgium,BEL


In [178]:
params = {
    "competitionId": "3",
    "limit": "15",
    "offset": "0",
    "optionalFields": "PLAYER,TEAM",
    "order": "DESC",
    "phase": "TOURNAMENT",
    "seasonYear": "2024",
    "stats": "matches_appearance,matches_win,matches_draw,matches_loss, passes_accuracy,passes_attempted,passes_completed,ball_possession,cross_accuracy,cross_attempted,cross_completed,free_kick,matches_appearance, goals,goals_scored_with_right,goals_scored_with_left,goals_scored_head,goals_scored_other,goals_scored_inside_penalty_area,goals_scored_outside_penalty_area,penalty_scored,matches_appearance, attacks,assists,corners,offsides,dribbling,recovered_ball,tackles,tackles_won,tackles_lost",
}

response = requests.get(
    "https://compstats.uefa.com/v1/team-ranking", params=params, headers=headers
)

In [179]:
stats_dfs = []

for r in response.json():
    team_id = r["team"]["associationId"]
    team_name = r["team"]["internationalName"]
    df = (
        pd.DataFrame(r["statistics"])
        .assign(team_name=team_name)
        .assign(team_id=team_id)
    )
    stats_dfs.append(
        df[["name", "value", "team_id", "team_name"]]
        .pivot(index=["team_id", "team_name"], columns="name", values="value")
        .reset_index()
    )

In [180]:
pd.concat(stats_dfs)

name,team_id,team_name,assists,attacks,ball_possession,corners,cross_accuracy,cross_attempted,cross_completed,dribbling,free_kick,goals,goals_scored_head,goals_scored_inside_penalty_area,goals_scored_other,goals_scored_outside_penalty_area,goals_scored_with_left,goals_scored_with_right,matches_appearance,matches_draw,matches_loss,matches_win,offsides,passes_accuracy,passes_attempted,passes_completed,penalty_scored,recovered_ball,tackles,tackles_lost,tackles_won
0,200047,Germany,3,63,68,5,31.82,22,7,14,8,5,0,3,0,2,1,4,1,0,0,1,4,94,697,657,1,37,12,10,2
0,200066,Italy,1,58,66,5,43.75,16,7,8,8,2,1,1,0,1,0,1,1,0,0,1,2,93,822,763,0,25,6,4,2
0,200039,England,1,34,53,1,42.86,7,3,18,19,1,1,1,0,0,0,0,1,0,0,1,1,91,600,544,0,44,13,6,7
0,200095,Netherlands,2,65,65,6,35,20,7,17,12,2,0,1,0,1,1,1,1,0,0,1,0,90,580,523,0,32,14,7,7
0,200122,Spain,3,33,46,5,36.37,11,4,19,13,3,0,3,0,0,2,1,1,0,0,1,2,86,455,392,0,36,8,7,1
0,200128,Switzerland,2,46,53,6,22.23,9,2,13,12,3,0,2,0,1,0,3,1,0,0,1,2,85,456,388,0,38,11,10,1
0,200035,Denmark,1,62,62,9,29.17,24,7,10,15,1,0,1,0,0,0,1,1,1,0,0,1,88,669,590,0,39,9,8,1
0,257163,Slovenia,0,24,38,5,30.77,13,4,7,10,1,0,0,0,1,1,0,1,1,0,0,3,78,331,259,0,47,15,12,3
0,200147,Serbia,0,33,47,2,10,20,2,5,9,0,0,0,0,0,0,0,1,0,1,0,0,89,512,454,0,29,20,16,4
0,256370,Croatia,0,32,54,0,28.58,14,4,12,15,0,0,0,0,0,0,0,1,0,1,0,0,88,528,467,0,29,11,7,4


---

## Exports

#### JSON

In [17]:
# df.to_json(
#     f"data/processed/NAME.json",
#     indent=4,
#     orient="records",
# )

#### CSV

In [18]:
# df.to_csv(
#     f"data/processed/NAME.csv", index=False
# )

#### GeoJSON

In [19]:
# gdf.to_file(
#     f"data/processed/NAME.geojson",
#     driver="GeoJSON",
# )